In [ ]:
!pip install 'aif360[LFR]' fairlearn gpflow


In [ ]:
cd / usr/local/lib/python3.7/dist-packages/aif360/data/raw/adult


In [ ]:
!wget https: // archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget https: // archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
!wget https: // archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test


In [64]:
import numpy as np
import tensorflow as tf
#from tensorflow import keras
import time
import pandas as pd

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels


import copy
import random


from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.metrics import ClassificationMetric
from aif360.sklearn.metrics import equal_opportunity_difference
from aif360.sklearn.datasets import standardize_dataset
from aif360.sklearn.preprocessing import Reweighing

import gpflow

from gpflow.utilities import print_summary, set_trainable
from gpflow.ci_utils import ci_niter
from gpflow.models import SVGP

import warnings
warnings.filterwarnings("ignore")

np.random.seed(0)
tf.random.set_seed(123)


In [127]:

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
adult_dataset = load_preproc_data_adult(['sex'])

train, test = adult_dataset.split([0.7], shuffle=True, seed=3)
#print("training data size", train.features.shape)
#print("dataset feature names", train.feature_names)

# data_preprocessing = Pipeline([("standardiser", StandardScaler()),
#("minmaxscaler", MinMaxScaler()),
# ]
# )


adult_income_data_train, adult_dict_train = train.convert_to_dataframe()
adult_income_data_test, adult_dict_test = test.convert_to_dataframe()

X_train, y_train = standardize_dataset(
    df=adult_income_data_train, prot_attr='sex', target='Income Binary')

X_test, y_test = standardize_dataset(
    df=adult_income_data_test, prot_attr='sex', target='Income Binary')

X_test = np.array(X_test)
y_test = np.array(y_test)
X_train = np.array(X_train)
y_train = np.array(y_train)


In [128]:
# x_train = train.features
# x_test = test.features
# # x_train = data_preprocessing.fit_transform(train.features)
# # x_test = data_preprocessing.fit_transform(test.features)
# y_train = train.labels.ravel()
# y_test = test.labels.ravel()

#training_data = (x_train, y_train)


# N = len(x_train)
# M = 400
print("Number of training datapoints:", len(X_train))
print("Number of features", len(X.columns))


Number of training datapoints: 34189
Number of features 18


## Model Training

### Setting up the model

In [5]:
class FairSVGP(SVGP):
    pass
    

In [135]:
class SVGP_classifier(ClassifierMixin, BaseEstimator):
    """ A multiclass classifier that uses SVGP (Hensman, 2013).

    Parameters
    ----------
    M : int, default=100
        The number of inducing points to be used (comuptation scales as O(M^3)).

    Attributes
    ----------
    X_ : ndarray, shape (n_samples, n_features)
        The input passed during :meth:`fit`.
    y_ : ndarray, shape (n_samples,)
        The labels passed during :meth:`fit`.
    classes_ : ndarray, shape (n_classes,)
        The classes seen at :meth:`fit`.
    """

    def __init__(self, M=100):
        self.M = M

        self.elbo_training_log = []
        self.fairness_log = []

    def run_adam(self, model, iterations, training_data, sample_weights, minibatch_size, learning_rate=0.001):
        """
        Utility function running the Adam optimizer

        :param model: GPflow model
        :param interations: number of iterations
        """
        # Create an Adam Optimizer action
        train_dataset = tf.data.Dataset.from_tensor_slices(training_data).repeat().shuffle(self.N)  # with weights for each datapoint

        logf = []

        # X, y = training_data
        # training_data = list(zip(*training_data))

        # # print(type(X))
        # # print(len(X))
        # # print(len(sample_weights))
        # # print(X[1])

        # def pick_minibatch(minibatch_size, sample_weights):
        #     p = np.array(sample_weights)/sum(sample_weights)
        #     minibatch = np.random.choice(
        #         training_data, size=minibatch_size, replace=False, p=p)
        #     return minibatch

        # print(pick_minibatch(minibatch_size, sample_weights))

        train_iter = iter(train_dataset.batch(minibatch_size))
        training_loss = model.training_loss_closure(train_iter, compile=True)

        optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

        @tf.function
        def optimization_step():
            optimizer.minimize(training_loss, model.trainable_variables)

        for step in range(iterations):
            optimization_step()
            if step % 10 == 0:
                elbo = -training_loss().numpy()
                logf.append(elbo/minibatch_size)
        return logf

    def fit(self, X, y, minibatch_size=130, total_iterations=10000, show_training_curve=False, show_training_summary=False, robustmax_epsilon=0.001, sample_weights=None):
        """A reference implementation of a fitting function for a classifier.

        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples.
        y : array-like, shape (n_samples,)
            The target values. An array of int.
        minibatch_size : int, 

        Returns
        -------
        self : object
            Returns self.
        """
        # Check that X and y have correct shape
        _, _ = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        print(type(X))

        self.X_ = X
        self.y_ = y
        self.N = len(y)
        self.C = len(self.classes_)

        if sample_weights == None:
            sample_weights = np.ones(shape=(self.N,))
        else:
            assert len(sample_weights) == self.N

        # RW = Reweighing()
        # # We obtain a set of weights for the training set, to use in scikit-learn.
        # _, reweights = RW.fit_transform(X, y)
        # reweights = np.array(reweights)

        kernel_multi = gpflow.kernels.Matern52() + gpflow.kernels.White(variance=0.01)

        # Robustmax Multiclass Likelihood

        invlink = gpflow.likelihoods.RobustMax(
            self.C,
            epsilon=robustmax_epsilon
        )  # Robustmax inverse link function

        likelihood_multi = gpflow.likelihoods.MultiClass(
            self.C,
            invlink=invlink
        )  # Multiclass likelihood

        Z = X[:self.M].copy()  # inducing inputs

        print(len(Z), "is the number of indvars")

        self.m = SVGP(
            kernel=kernel_multi,
            likelihood=likelihood_multi,
            inducing_variable=Z,
            num_latent_gps=self.C,
            whiten=True,
            q_diag=True,
            mean_function=gpflow.mean_functions.Constant(c=1.0),
            q_mu=np.ones(self.M)
        )

        #set_trainable(self.m.kernel.kernels[0].variance, False)
        #set_trainable(self.m.inducing_variable, False)

        self.mb_size = minibatch_size
        training_data = (X, y)
        maxiter = ci_niter(total_iterations)
        self.elbo_training_log = self.run_adam(
            self.m,
            maxiter,
            training_data,
            sample_weights,
            self.mb_size)

        if show_training_curve == True:
            plt.plot(np.linspace(0, total_iterations, len(
                self.elbo_training_log)), self.elbo_training_log)
            plt.xlabel("Iteration")
            plt.ylabel("ELBO per data point")
            plt.title("Training curve")
            plt.show()

        if show_training_summary == True:
            print_summary(self.m, fmt="notebook")

        # Return the classifier
        return self

    def predict(self, X_test):
        """ A reference implementation of a prediction for a classifier.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        y : ndarray, shape (n_samples,)
            The label for each sample is the label of the closest sample
            seen during fit.
        """
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])

        # Input validation
        X_test = check_array(X_test)

        mu, _ = self.m.predict_y(X_test)
        predictions = []
        for probs in mu:
            prediction = np.argmax(probs)
            predictions.append(prediction)
        return predictions


### Training the models

In [25]:
from fairlearn.metrics import true_positive_rate


def tpr_differ(y_test, predictions):
    female_test = []
    male_test = []
    female_pred = []
    male_pred = []
    for i, index in enumerate(list(y_test.index)):
        if index[1] == 0:
            female_test.append(y_test[index])
            female_pred.append(predictions[i])
        if index[1] == 1:
            male_test.append(y_test[index])
            male_pred.append(predictions[i])
    tpr_male = true_positive_rate(male_test, male_pred)
    tpr_female = true_positive_rate(female_test, female_pred)
    return tpr_female-tpr_male


def positives_ratio(y_test, predictions):
    test_positives = list(y_test).count(1)
    pred_positives = predictions.count(1)
    return pred_positives/test_positives


In [136]:
pos_clf = SVGP_classifier(M=100)
pos_clf.fit(X_train, y_train, minibatch_size=100, total_iterations=100,
            show_training_curve=True, show_training_summary=True)
preds = pos_clf.predict(X_test)

print(accuracy_score(y_test, preds))
print(tpr_differ(y_test, preds))
print(positives_ratio(y_test, preds))


<class 'numpy.ndarray'>
100 is the number of indvars


ValueError: in user code:

    /var/folders/hf/nwz23j_d1dx_8v_yxrw0v_vh0000gn/T/ipykernel_38652/1950641272.py:60 optimization_step  *
        optimizer.minimize(training_loss, model.trainable_variables)
    /Users/jacob/Programming/GPflow/gpflow/models/training_mixins.py:99 training_loss  *
        return self._training_loss(data)
    /Users/jacob/Programming/GPflow/gpflow/models/model.py:56 _training_loss  *
        return -(self.maximum_log_likelihood_objective(*args, **kwargs) + self.log_prior_density())
    /Users/jacob/Programming/GPflow/gpflow/models/svgp.py:139 maximum_log_likelihood_objective  *
        return self.elbo(data)
    /Users/jacob/Programming/GPflow/gpflow/models/svgp.py:147 elbo  *
        kl = self.prior_kl()
    /Users/jacob/Programming/GPflow/gpflow/models/svgp.py:135 prior_kl  *
        self.inducing_variable, self.kernel, self.q_mu, self.q_sqrt, whiten=self.whiten
    /Users/jacob/opt/anaconda3/lib/python3.8/site-packages/multipledispatch/dispatcher.py:278 __c

In [ ]:
#classify_me = SVGP_classifier(400)
#classify_me.fit(X_train, y_train, minibatch_size=100, total_iterations=100, show_training_curve=True, show_training_summary=True)

RW = Reweighing()
# We obtain a set of weights for the training set, to use in scikit-learn.
_, reweights = RW.fit_transform(X_train, y_train)

print(sum(reweights))

# X_test = training_data[0]
        # y_test = training_data[1]
        # y_pred = self.predict(X_test)

        # @tf.function
        # def eq_opp_fairness():
        #     from fairlearn.metrics import true_positive_rate
        #     female_test = []
        #     male_test = []
        #     female_pred = []
        #     male_pred = []
        #     for i, index in enumerate(list(y_test)):
        #         if index[1]==0:
        #             female_test.append(y_test[index])
        #             female_pred.append(y_pred[i])
        #         if index[1]==1:
        #             male_test.append(y_test[index])
        #             male_pred.append(y_pred[i])
        #     tpr_male = true_positive_rate(male_test, male_pred)
        #     tpr_female = true_positive_rate(female_test, female_pred)
        #     return tpr_female-tpr_male


34188.99999999158
34189


In [ ]:
# mb_sizes = list(np.linspace(10, 500, 50, dtype=int))
# print(mb_sizes)

epsilons = np.linspace(0.01, 0.03, 10, endpoint=True)
epsilons = [[epsilon] for epsilon in epsilons]


scoring_metrics = {"accuracy": "accuracy", "equal_opportunity_difference": make_scorer(
    tpr_differ), "positives_ratio": make_scorer(positives_ratio)}

score_log = []

with tf.device('/device:GPU:0'):
    for epsilon in epsilons:
        print(epsilon)
        svgpclass = SVGP_classifier(400)
        scores = cross_validate(svgpclass, X_train, y_train, scoring=scoring_metrics, cv=5, fit_params={
                                "robustmax_epsilon": epsilon, "total_iterations": 10000})
        print(
            "RobustMax epsilon:", epsilon, "\n",
            "Accuracy:", np.mean(scores["test_accuracy"]),
            "     Fairness:", np.mean(
                scores["test_equal_opportunity_difference"]),
            "     Positives ratio:", np.mean(scores["test_positives_ratio"]))
        score_log.append(scores)


[0.01]
RobustMax epsilon: [0.01] 
 Accuracy: 0.8013687502788687      Fairness: -0.43390015389550785      Positives ratio: 0.5566692342748628
[0.012222222222222221]
RobustMax epsilon: [0.012222222222222221] 
 Accuracy: 0.8013395019606468      Fairness: -0.4354704608191339      Positives ratio: 0.5594520533916203
[0.014444444444444444]
RobustMax epsilon: [0.014444444444444444] 
 Accuracy: 0.8007545355962129      Fairness: -0.4269792499407056      Positives ratio: 0.5473584724852618
[0.016666666666666666]
RobustMax epsilon: [0.016666666666666666] 
 Accuracy: 0.8007252872779912      Fairness: -0.4222817410439085      Positives ratio: 0.5394987385070273
[0.018888888888888886]
RobustMax epsilon: [0.018888888888888886] 
 Accuracy: 0.8008422805508779      Fairness: -0.4256852168709716      Positives ratio: 0.5448176376395268
[0.02111111111111111]
RobustMax epsilon: [0.02111111111111111] 
 Accuracy: 0.801427246915312      Fairness: -0.4279086119289768      Positives ratio: 0.5462686654508933
[0

In [13]:
accuracies = []
fairnesses = []
ratios = []


for score in score_log:
    fairness = np.mean(score["test_equal_opportunity_difference"])
    accuracy = np.mean(score["test_accuracy"])
    ratio = np.mean(score["test_positives_ratio"])
    accuracies.append(accuracy)
    fairnesses.append(fairness)
    ratios.append(ratio)

plt.figure(figsize=12)
plt.plot(epsilons, accuracies)
plt.xlabel("Trade-off parameter (epsilon)")
plt.ylabel("Accuracy")
plt.hlines(accuracy_score(y_test, np.zeros(len(y_test))),
           epsilons[0], epsilons[-1], linestyles="dashed", label="base accuracy", alpha=0.3)

plt.show()

plt.plot(fairnesses)
plt.show()

plt.plot(ratios)
plt.show()


NameError: ignored

In [ ]:


# elbo_training_logs = []
# models = []


# with tf.device('/device:GPU:0'):
#     for mb_size in mb_sizes:
#         model = copy.deepcopy(m)
#         elbo_training_log = run_adam(model, maxiter, training_data, mb_size)
#         elbo_training_logs.append(np.array(elbo_training_log)/mb_size)
#         models.append(model)


# for log in elbo_training_logs:
#     plt.plot(range(0, total_iterations, 10), log)
#     plt.show()


In [ ]:


# def calculate_test_accuracy(model, x_test, y_test):
#     predictions = predict_targets(model, x_test)
#     total_correct = 0
#     for i in range(len(x_test)):
#         if predictions[i]==y_test[i]:
#             total_correct += 1

#     return total_correct/len(predictions)


# def logistic_regression_accuracy(training_data, x_test, y_test):
#     model = LogisticRegression()
#     model.fit(*training_data)
#     predictions = model.predict(x_test)
#     return accuracy_score(y_test, predictions)

# def mlp_accuracy(training_data, x_test, y_test):
#     model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
#     model.fit(*training_data)
#     predictions = model.predict(x_test)
#     return accuracy_score(y_test, predictions)


# fairness_scores = []
# test_accuracies = []

# for i in range(len(models)):
#     test_accuracy = calculate_test_accuracy(models[i], x_test, y_test)
#     print(f"SVGP accuracy (minibatch = {mb_sizes[i]}):", test_accuracy)
#     test_accuracies.append(test_accuracy)
#     predictions = predict_targets(models[i], x_test)
#     test_pred = test.copy()
#     test_pred.labels = np.array(predictions)
#     fairness_metric = ClassificationMetric(
#         test,
#         test_pred,
#         unprivileged_groups=unprivileged_groups,
#         privileged_groups=privileged_groups
#         )
#     fairness_scores.append(fairness_metric.equal_opportunity_difference())
# print("Logistic Regression accuracy:", logistic_regression_accuracy(training_data, x_test, y_test))
# print("MLP accuracy:", mlp_accuracy(training_data, x_test, y_test))
